<a href="https://colab.research.google.com/github/AndAmir/Covid19-model-colab/blob/master/Covid19_precondition_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
#!pip install tensorflow-gpu
#!pip install mlxtend==0.17.0

In [20]:
import tensorflow as tf
import pandas as pd
import datetime as dt
import numpy as np

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [21]:
'''
df = pd.read_csv("/content/covid.csv")
df = df[df.icu != 97.0]
df.insert(6,"symptomEntry_Delta",0)

df.reset_index(drop=True)
for a, b, i in zip(df.date_symptoms, df.entry_date,range(121913)):
  a = dt.date(2020, int(b[3:5]),int(b[:2])) - dt.date(2020, int(a[3:5]),int(a[:2]))
  df.iloc[i,df.columns.get_loc('symptomEntry_Delta')] = a.days
df.pop('id')

df['icu'].value_counts()
df = df[df.icu != 99.0]
df.reset_index(drop=True)
df.to_csv('covid_edited.csv')
'''

'\ndf = pd.read_csv("/content/covid.csv")\ndf = df[df.icu != 97.0]\ndf.insert(6,"symptomEntry_Delta",0)\n\ndf.reset_index(drop=True)\nfor a, b, i in zip(df.date_symptoms, df.entry_date,range(121913)):\n  a = dt.date(2020, int(b[3:5]),int(b[:2])) - dt.date(2020, int(a[3:5]),int(a[:2]))\n  df.iloc[i,df.columns.get_loc(\'symptomEntry_Delta\')] = a.days\ndf.pop(\'id\')\n\ndf[\'icu\'].value_counts()\ndf = df[df.icu != 99.0]\ndf.reset_index(drop=True)\ndf.to_csv(\'covid_edited.csv\')\n'

In [22]:
df = pd.read_csv('/content/covid_edited.csv')

In [38]:
df.drop(columns=['date_symptoms', 'entry_date','date_died','patient_type','contact_other_covid'], inplace= True)

In [26]:
df.drop(df.loc[df['diabetes']==98].index, inplace=True)
df.drop(df.loc[df['inmsupr']==98].index, inplace=True)
df.drop(df.loc[df['asthma']==98].index, inplace=True)
df.drop(df.loc[df['copd']==98].index, inplace=True)
df.drop(df.loc[df['hypertension']==98].index, inplace=True)
df.drop(df.loc[df['other_disease']==98].index, inplace=True)
df.drop(df.loc[df['obesity']==98].index, inplace=True)
df.drop(df.loc[df['renal_chronic']==98].index, inplace=True)
df.drop(df.loc[df['tobacco']==98].index, inplace=True)
df.drop(df.loc[df['cardiovascular']==98].index, inplace=True)

In [32]:
df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

In [40]:
df = df.reset_index(drop=True)

In [ ]:
df['covid_res'].value_counts()

In [42]:
df.to_csv('covid_modelReady.csv')

In [ ]:
feature_columns = []

# numeric cols
for header in ['sex', 'symptonEntry_Delta', 'intubed','pneumonia','age','diabetes','copd','asthma','imsupr','hypertension','other_disease','cardiovascular','renal_chronic','tobacco','']:
  feature_columns.append(feature_column.numeric_column(header))